In [25]:
import pickle as pk
import numpy as np
from keras.datasets import mnist
from tqdm import tqdm

In [26]:
class utils():
    def padder(self, data, pad_length):
        im = data.copy()
        m = im.shape[0]
        n = im.shape[1]
        M, N = m + 2*pad_length, n + 2*pad_length 
        im2 = np.zeros((M,N))
        im2[pad_length:m+pad_length , pad_length:n+pad_length] = im[:,:]
        
        return im2
    
    def Conv2D(self, inputs, weights, bias, padding, K, F, stride =1):
        C, W, H = inputs.shape
        WW = (W - K)//stride + 1
        HH = (H - K)//stride + 1

        feature_maps = np.zeros((F, WW, HH))

        for f in range(F):
            for w in range(WW):
                for h in range(HH):
                    # ic(f, w, h, K, weights[f, :, :, :].shape, bias[f].shape, inputs[:, w:w+K, h:h+K].shape)
                    wi = w * stride
                    wj = wi + K 
                    hi = h * stride
                    hj = hi + K 
                    feature_maps[f,wi,hi]=np.sum(inputs[:,wi:wj,hi:hj]*weights[f,:,:,:])+bias[f]

        return feature_maps

    def AvgPool(self, data, pool_size, stride):
        C, W, H = data.shape

        new_width = (W - pool_size)//stride + 1
        new_height = (H - pool_size)//stride + 1

        out = np.zeros((C, new_width, new_height))

        for c in range(C):
            for w in range(new_width):
                for h in range(new_height):
                    out[c, w, h] = np.mean(data[c, w*stride:w*stride+pool_size, h*stride:h*stride+pool_size])
  
        return out

    def softmax(self, x):
        denom = np.sum(np.exp(x))
        y = np.exp(x)/denom;
        return y

    def sigmoid(self, x):
        return 1/(1+np.exp(-x))

    def Tanh(self, x):
        a = 1.7159 
        s = 2/3
        return a*np.tanh(s*x)

    def backTanh(self, x):
        a = 1.7159 
        s = 2/3
        return a*s*(1-np.tanh(s*x)**2)
    
    def ReLU(self, x):
        return np.max(x,0)
    
    def normal(self, data):
        padded = self.padder(data,2)
    
    def vanilla(self, data, weights, bias, activation = 'Tanh'):
        if activation == 'Tanh':
            # ic(np.transpose(weights).shape, data[:,0].shape)
            # ic(data)
            return self.Tanh(np.dot(np.transpose(weights),data) + bias)
        elif activation == 'sigmoid':
            return  self.sigmoid(np.dot(np.transpose(weights),data) + bias)

In [27]:
util = utils()
class Layers():
    class CONV():
        def __init__(self, inputs_channel, num_filters, kernel_size):
          self.F = num_filters
          self.K = kernel_size
          self.C = inputs_channel

          self.weights = 2*(np.random.rand(self.F,self.C, self.K, self.K)-0.5)
          self.bias = 2*(np.random.rand(self.F, 1)-0.5)

        def forward(self, inputs, padding, stride):
            C = inputs.shape[0]
            W = inputs.shape[1]+2*padding
            H = inputs.shape[2]+2*padding

            self.inputs = np.zeros((C, W, H))

            for c in range(inputs.shape[0]):
                self.inputs[c,:,:] = util.padder(inputs[c,:,:], padding)

            return util.Conv2D(self.inputs, self.weights, self.bias, padding, self.K, self.F, stride)
          
        def backward(self, dy, stride, learning_rate):
            dy = util.backTanh(dy)

            C, W, H = self.inputs.shape

            dx = np.zeros(self.inputs.shape)
            dw = np.zeros(self.weights.shape)
            db = np.zeros(self.bias.shape)

            if len(dy.shape)==2:
              dy = np.array([dy])
            F, W, H = dy.shape

            for f in range(F):
                for w in range(0, W-self.K, stride):
                    for h in range(0, H-self.K, stride):
                        dw[f,:,:,:]+=dy[f,w,h]*self.inputs[:,w:w+self.K,h:h+self.K]
                        dx[:,w:w+self.K,h:h+self.K]+=dy[f,w,h]*self.weights[f,:,:,:]

            for f in range(F):
                db[f] = np.sum(dy[f, :, :])

            self.weights -= learning_rate * dw
            self.bias -= learning_rate * db

            return dx
    
    class POOL():
        def __init__(self, pool_size):
            self.pool = pool_size
        
        def forward(self, data, stride):
            self.inputs = data
            return util.AvgPool(data,self.pool,stride)
        # change later
        def backward(self, dy):
            C, W, H = self.inputs.shape
            dx = np.zeros(self.inputs.shape)
            
            for c in range(C):
                for w in range(0, W, self.pool):
                    for h in range(0, H, self.pool):
                        st = np.argmax(self.inputs[c,w:w+self.pool,h:h+self.pool])
                        (idx, idy) = np.unravel_index(st, (self.pool, self.pool))
                        dx[c, w+idx, h+idy] = dy[c, w//self.pool, h//self.pool]
            return dx
    
    class DENSE():
        def __init__(self, num_inputs, num_outputs, act):
            self.weights = 2*(np.random.rand(num_inputs, num_outputs)-0.5)
            self.bias = 2*(np.random.rand(num_outputs, 1)-0.5)
            self.act=act
        
        def forward(self, data):
            self.inputs = data
            if self.act == 'tanh':
              return util.Tanh(util.vanilla(data,self.weights,self.bias))
            elif self.act == 'softmax':
              self.out = util.softmax(util.vanilla(data,self.weights,self.bias)[:,0])
              return self.out

        def backward(self, dy, learning_rate):
            # print("blah",dy.shape)
            if self.act == 'tanh':
                dy = util.backTanh(dy).T
                dw = dy.dot(self.inputs.T)
                db = np.sum(dy, axis=1, keepdims=True)
                dx = np.dot(dy.T, self.weights.T)
            elif self.act=='softmax':
                # print("bleh",(self.out.T - dy.reshape(dy.shape[0],1)).shape)
                # print(self.out.T)
                # print(dy.reshape(dy.shape[0],1))
                # print(self.out.T - dy.reshape(dy.shape[0],1))
                # dy = self.out.T - dy.reshape(dy.shape[0],1)
                dy = self.out.T.reshape(dy.shape[0],1) - dy.reshape(dy.shape[0],1)
                dw = dy.dot(self.inputs.T)
                db = np.sum(dy, axis=1, keepdims=True)
                dx = np.dot(dy.T, self.weights.T)

            # print("blah",dy.shape)
            # if dy.shape[0] == self.inputs.shape[0]:
            #     dy = dy.T
            # print("blah",dy.shape)
            # dw = dy.dot(self.inputs)
            # db = np.sum(dy, axis=1, keepdims=True)
            # dx = np.dot(dy.T, self.weights.T)

            self.weights -= learning_rate * dw.T
            self.bias -= learning_rate * db

            return dx

In [28]:
def store_pickle(data, loc):
    try:
        pk.dump(data, open(loc, "wb"))
        return 0
        
    except Exception as e:
        return e

In [29]:
d = {0: [1, 1, 1, 0, 0, 0],
 1: [0, 1, 1, 1, 0, 0],
 2: [0, 0, 1, 1, 1, 0],
 3: [0, 0, 0, 1, 1, 1],
 4: [1, 0, 0, 0, 1, 1],
 5: [1, 1, 0, 0, 0, 1],
 6: [1, 1, 1, 1, 0, 0],
 7: [0, 1, 1, 1, 1, 0],
 8: [0, 0, 1, 1, 1, 1],
 9: [1, 0, 0, 1, 1, 1],
 10: [1, 1, 0, 0, 1, 1],
 11: [1, 1, 1, 0, 0, 1],
 12: [1, 1, 0, 1, 1, 0],
 13: [0, 1, 1, 0, 1, 1],
 14: [1, 0, 1, 1, 0, 1],
 15: [1, 1, 1, 1, 1, 1]}

In [30]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()
print('X_train: ' + str(train_X.shape))
print('Y_train: ' + str(train_y.shape))
print('X_test:  ' + str(test_X.shape))
print('Y_test:  ' + str(test_y.shape))

X_train: (60000, 28, 28)
Y_train: (60000,)
X_test:  (10000, 28, 28)
Y_test:  (10000,)


In [31]:
layers = Layers()
C1 = layers.CONV(1,6,5)
S2 = layers.POOL(2)
C3 = layers.CONV(6,16,5)
S4 = layers.POOL(2)
C5 = layers.CONV(16,120,5)
F6 = layers.DENSE(120,84,'tanh')
F7 = layers.DENSE(84,10,'softmax')
def forward_pass(im):
  fmap1 = util.Tanh(C1.forward(np.array([im]),2,1))
  # print("C1 Done", fmap1.shape)
  fmap2 = S2.forward(fmap1,2)
  # print("S2 Done", fmap2.shape)
  fmap3 = util.Tanh(C3.forward(fmap2,0,1))
  # print("C3 Done", fmap3.shape)
  fmap4 = S4.forward(fmap3,2)
  # print("S4 Done", fmap4.shape)
  fmap5 = util.Tanh(C5.forward(fmap4,0,1))
  # print("C5 Done", fmap5.shape)
  fmap6 = F6.forward(fmap5[:,0])
  # print("F6 Done", fmap6.shape)
  fmap7 = F7.forward(fmap6)
  return [fmap1, fmap2, fmap3, fmap4, fmap5, fmap6, fmap7]


all_layers = [C1, S2, C3, S4, C5, F6, F7]

In [32]:
output = forward_pass(train_X[0])

In [33]:
# print(output[-1].reshape(10,1),output[-1].reshape(10,1).T)

# Training



In [34]:
def one_hot(x):
    temp = np.zeros(10)
    temp[x] = 1
    return temp

In [35]:
def train(epochs):
    for epoch in range(epochs):
        print("Epoch:",epoch)
        for i, im in tqdm(enumerate(train_X[:2000])):
            # if i%100==0:
              # print(i, end=" ")
            label = one_hot(train_y[i])
            fmaps = forward_pass(im)
            t = F7.backward(fmaps[-1], learning_rate = 0.05)
            t = F6.backward(t, learning_rate = 0.05)
            t = C5.backward(t, stride = 1, learning_rate = 0.05)
            t = S4.backward(t)
            t = C3.backward(t, stride = 1, learning_rate = 0.05)
            t = S2.backward(t)
            t = C1.backward(t, stride = 1, learning_rate = 0.05)


In [ ]:
train(5)

Epoch: 0


469it [01:45,  4.46it/s]

In [ ]:
def test(ind, to_print=True):
    val = forward_pass(train_X[ind])
    if to_print:
      print("True Label:", train_y[ind], "Predicted:", np.argmax(val[-1]))
    return train_y[ind], np.argmax(val[-1])

In [ ]:
acc = 0
for i in tqdm(range(1000,1500)):
  x,y = test(i,to_print=False)
  if x==y:
    acc += 1
acc/=500
acc